In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings,OpenAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
import datetime

In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

<h3> Partial With Strings </h3>

In [4]:
prompt=PromptTemplate.from_template(template="{foo}{bar}")

In [5]:
prompt

PromptTemplate(input_variables=['bar', 'foo'], input_types={}, partial_variables={}, template='{foo}{bar}')

In [6]:
partialPrompt=prompt.partial(foo="foo")

In [7]:
partialPrompt

PromptTemplate(input_variables=['bar'], input_types={}, partial_variables={'foo': 'foo'}, template='{foo}{bar}')

In [8]:
partialPrompt.format(bar='bar')

'foobar'

In [9]:
chain=RunnablePassthrough()|partialPrompt|llmGemini|StrOutputParser()

In [10]:
chain.invoke(input={'bar':"bar"})

'You got it! "Foobar" is a classic placeholder name, often used in programming examples or documentation when a name is needed but the actual name doesn\'t matter.\n\nIs there anything else I can help you with regarding "foobar" or anything else related to programming or general knowledge?'

In [12]:
# Initialize the partial with partial variables

In [11]:
prompt=PromptTemplate(template="{foo}{bar}",input_variables=['bar'],partial_variables={"foo":"foo"})

In [12]:
prompt

PromptTemplate(input_variables=['bar'], input_types={}, partial_variables={'foo': 'foo'}, template='{foo}{bar}')

In [38]:
chain=RunnablePassthrough()|prompt|llmGemini|StrOutputParser()

In [39]:
chain.invoke(input={'bar':"bar"})

'"Foobar" is a placeholder word often used in programming, testing, and other technical contexts. It has no specific meaning and is typically used to represent:\n\n* **Dummy data:**  A placeholder for real data that will be used later.\n* **Testing:**  A placeholder for a function or value during testing.\n* **Example code:** A generic term for code that demonstrates a concept or functionality.\n\nIt\'s like a "lorem ipsum" for code! \n'

<h3>Partial With Functions</h3>

In [13]:
def _getDateTime():
    now=datetime.datetime.now()
    return now.strftime("%m/%d/%Y %H:%M:%S")

In [14]:
_getDateTime()

'05/21/2025 00:45:11'

In [15]:
prompt=PromptTemplate(
    template="Tell me an {adjective} joke for the day {date}",
    input_variables=['adjective','date']
)

In [16]:
prompt

PromptTemplate(input_variables=['adjective', 'date'], input_types={}, partial_variables={}, template='Tell me an {adjective} joke for the day {date}')

In [17]:
partialPrompt=prompt.partial(date=_getDateTime)

In [18]:
partialPrompt

PromptTemplate(input_variables=['adjective'], input_types={}, partial_variables={'date': <function _getDateTime at 0x000002BC5A201EE0>}, template='Tell me an {adjective} joke for the day {date}')

In [19]:
partialPrompt.format(adjective="funny")

'Tell me an funny joke for the day 05/21/2025 00:46:46'

In [20]:
chain=RunnablePassthrough()|partialPrompt|llmGemini|StrOutputParser()

In [21]:
chain.invoke(input={'adjective':"funny"})

"Okay, here's a joke for you, set to be funny on May 21st, 2025 at 12:47:02 AM:\n\nWhy did the AI cross the road on May 21st, 2025?\n\n... Because it finally understood the CAPTCHA! And, also, to tell you that by this point in the future, knock-knock jokes are considered *ancient* history. Get with the times, human!"

In [55]:
# Initialize the Prompt with Partialized Variable

In [56]:
prompt=PromptTemplate(
    template="Tell me an {adjective} joke for the day {date}",
    input_variables=['adjective'],
    partial_variables={'date':_getDateTime}
)

In [57]:
prompt

PromptTemplate(input_variables=['adjective'], input_types={}, partial_variables={'date': <function _getDateTime at 0x00000237C6619F80>}, template='Tell me an {adjective} joke for the day {date}')

In [58]:
prompt.format(adjective="horror")

'Tell me an horror joke for the day 10/19/2024 21:21:38'

In [59]:
chain=RunnablePassthrough()|prompt|llmGemini|StrOutputParser()

In [60]:
chain.invoke(input={'adjective':"funny"})

"I can't tell you a joke specific to the date and time you provided.  I don't have access to real-time information like that. \n\nHowever, I can tell you a funny joke:\n\nWhy don't scientists trust atoms? \n\nBecause they make up everything! \n\nLet me know if you'd like to hear another joke! 😊 \n"